In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install openpyxl

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle as cPickle
import pandas as pd
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from keras.layers import LSTM, Dense, TimeDistributed, Bidirectional
import sklearn.metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras import Model
from keras.layers import Dense,Embedding,LSTM,Dropout,Bidirectional
from keras import regularizers
from keras import models
from openpyxl import Workbook

In [ ]:
path_data = "/content/drive/MyDrive/SuccKD_MultiSpecies/Data/"
path_model = "/content/drive/MyDrive/SuccKD_MultiSpecies/Model/"
path_result = "/content/drive/MyDrive/SuccKD_MultiSpecies/Result_cv/"

In [ ]:
file_train = "Train_T.Gondii_succ.csv"

In [ ]:
# Build an amino acid dictionary
def twoTupleDic1():
    AA_list_sort = ['G','A','V','L','I','M','P','F','W','S','T','N','Q','Y','C','K','R','H','D','E','X']

    AA_dict = {}
    numm = 1
    for i in AA_list_sort:
        AA_dict[i] = numm
        numm += 1
    return AA_dict
def ProSentence(pro, K):
	sentence = ""
	length = len(pro)
	for i in range(length - K + 1):
		sentence += pro[i: i + K] + " "
    #delete extra space
	sentence = sentence[0 : len(sentence) - 1]
	return sentence
k =1#1-gram
word_index1 = twoTupleDic1()
vocab_size = len(word_index1)


In [ ]:
#  Load train data and pre-processing
df_train =pd.read_csv(path_data+file_train, delimiter= ',')
texts_train =[]
for i in df_train['Peptide_WS_33']:
  temp = ProSentence(i,k)
  texts_train.append(temp)
df_train['k_mer'] =texts_train
train_sequences = []
for each in texts_train:
    each_index_list = []
    each = each.split(' ')
    for i in each:
        each_index_list.append(word_index1[i])
    train_sequences.append(each_index_list)
# Tokenizer
data_token = []
for i in df_train['k_mer']:
   data_token.append(i.split())
MAX_SEQUENCE_LENGTH = len(data_token[1])

Xtrain = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
ytrain = np.array(df_train['Label'])
ytrain = np.array(ytrain)
#one-hot encoding on the labels
lb = LabelBinarizer()
ytrain= lb.fit_transform(ytrain)
ytrain = to_categorical(ytrain)
ytrain.shape


(846, 2)

In [ ]:
TIME_STEPS = MAX_SEQUENCE_LENGTH
INPUT_SIZE = 300

In [ ]:
# KD2_succi
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras import models, layers, losses

# Teacher 1
def create_cnn_model():
    model = models.Sequential()
    model.add(layers.Embedding(vocab_size+1, 300, input_length=MAX_SEQUENCE_LENGTH,trainable=True))
    model.add(layers.Dropout(0.4))
    model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Dropout(0.4))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(2, activation='softmax'))  # Binary classification (softmax for distillation)
    return model

# Teacher 2
def create_bilstm_model():
    model = models.Sequential()
    model.add(layers.Embedding(vocab_size+1, 300, input_length=MAX_SEQUENCE_LENGTH,trainable=True))
    model.add(layers.Bidirectional(layers.LSTM(units=32, return_sequences=True), input_shape=(TIME_STEPS, INPUT_SIZE)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.2))
    model.add(layers.Bidirectional(layers.LSTM(units=32, return_sequences=True), input_shape=(TIME_STEPS, INPUT_SIZE)))
    model.add(layers.BatchNormalization())
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(2, activation='softmax'))  # Binary classification (softmax for distillation)
    return model
# Student
def create_student():
    model = models.Sequential()
    model.add(layers.Embedding(vocab_size+1, 300, input_length=MAX_SEQUENCE_LENGTH,trainable=True))
    model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Dropout(0.2))
    model.add(layers.BatchNormalization())
    model.add(layers.Bidirectional(layers.LSTM(units=32, return_sequences=True), input_shape=(TIME_STEPS, INPUT_SIZE)))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(2, activation='softmax'))  # Binary classification (softmax for distillation)
    return model
#  KD2_succi: Knowlege distillation
import tensorflow as tf
from tensorflow.keras import optimizers

class Distiller(tf.keras.Model):
    def __init__(self, teacher1, teacher2, student, alpha=0.5, temperature=3.0):
        super(Distiller, self).__init__()
        self.teacher1 = teacher1
        self.teacher2 = teacher2
        self.student = student
        self.alpha = alpha
        self.temperature = temperature

        # Optimizer for the student
        self.student_optimizer = AdamW(learning_rate=0.0001)

    def compile(self, metrics, student_loss_fn, distillation_loss_fn):
        super(Distiller, self).compile(metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn

    def call(self, inputs):
        """
        This method defines how the model processes inputs.
        Args:
        - inputs: Input data (Xtest_student or any other data).

        Returns:
        - Output logits from the student model.
        """
        student_logits = self.student(inputs, training=False)
        return student_logits

    def train_step(self, data):
        x, y = data

        # Sequentially forward pass through teachers
        teacher1_logits = self.teacher1(x, training=False)
        teacher2_logits = self.teacher2(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass through student
            student_logits = self.student(x, training=True)

            # Compute the task loss (for hard labels)
            student_loss = self.student_loss_fn(y, student_logits)

            # Compute distillation losses (for soft labels)
            distill_loss1 = self.distillation_loss_fn(
                tf.nn.softmax(teacher1_logits / self.temperature, axis=1),
                tf.nn.softmax(student_logits / self.temperature, axis=1)
            ) * (self.temperature ** 2)

            distill_loss2 = self.distillation_loss_fn(
                tf.nn.softmax(teacher2_logits / self.temperature, axis=1),
                tf.nn.softmax(student_logits / self.temperature, axis=1)
            ) * (self.temperature ** 2)

            # Combined loss
            total_loss = self.alpha * student_loss + (1 - self.alpha) * (distill_loss1 + distill_loss2)

        # Compute gradients and update student model
        grads = tape.gradient(total_loss, self.student.trainable_variables)
        self.student_optimizer.apply_gradients(zip(grads, self.student.trainable_variables))

        # Update metrics for the student
        self.compiled_metrics.update_state(y, student_logits)

        # Return results
        results = {
            'student_loss': student_loss,
            'distill_loss1': distill_loss1,
            'distill_loss2': distill_loss2
        }
        results.update({m.name: m.result() for m in self.metrics})
        return results

    def test_step(self, data):
        x, y = data
        student_logits = self.student(x, training=False)

        # Compute the task loss (for hard labels)
        student_loss = self.student_loss_fn(y, student_logits)

        # Update metrics for the student
        self.compiled_metrics.update_state(y, student_logits)

        return {
            'student_loss': student_loss,
            **{m.name: m.result() for m in self.metrics}
        }



In [ ]:
# Transfer learning from Generic dl set to species
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from tensorflow.keras.optimizers import AdamW
from keras import models
k = 10
kf = KFold(n_splits=k, shuffle=True, random_state=42)
all_metrics = []
confusion_matrices = []
#  K-fold cross-validation
for fold, (train_index, val_index) in enumerate(kf.split(Xtrain)):
    print(f"Training on fold {fold + 1}/{k}...")
    X_train_fold, X_val_fold = Xtrain[train_index], Xtrain[val_index]
    y_train_fold, y_val_fold = ytrain[train_index], ytrain[val_index]
    student_fold = create_student()
    teacher1_fold = models.load_model(path_model +"model_teacher21.h5") # Sử dụng teacher giống nhau trên cùng loài
    teacher2_fold = models.load_model(path_model +"model_teacher22.h5")
    #distiller fold
    distiller_fold = Distiller(teacher1=teacher1_fold, teacher2=teacher2_fold, student=student_fold, alpha=0.5, temperature=10)
    distiller_fold.compile(
        student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=["categorical_accuracy", "AUC"],
        distillation_loss_fn=keras.losses.KLDivergence(),
    )

    # Train model
    history = distiller_fold.fit(
        X_train_fold, y_train_fold,
        validation_data=(X_val_fold, y_val_fold),
        batch_size=16,
        epochs=50,

    )
    results = distiller_fold.evaluate(X_val_fold, y_val_fold)
    all_metrics.append(results)
    y_pred_fold = student_fold.predict(X_val_fold)
    y_pred_classes = np.argmax(y_pred_fold, axis=1)
    y_true_classes = np.argmax(y_val_fold, axis=1)
    # confusion matrix and save TP, TN, FP, FN
    cm = confusion_matrix(y_true_classes, y_pred_classes)
    tn, fp, fn, tp = cm.ravel()
    confusion_matrices.append({"TP": tp, "TN": tn, "FP": fp, "FN": fn})
    print(f"Fold {fold + 1} - TP: {tp}, TN: {tn}, FP: {fp}, FN: {fn}")

# Average
avg_metrics = np.mean(all_metrics, axis=0)
print(f"\nAverage Metrics over {k}-Fold Cross Validation:")
for metric_name, avg_metric in zip(distiller_fold.metrics_names, avg_metrics):
    print(f"{metric_name}: {avg_metric:.4f}")
# Show TP, TN, FP, FN cho từng fold
for fold, cm in enumerate(confusion_matrices):
    print(f"Fold {fold + 1} - TP: {cm['TP']}, TN: {cm['TN']}, FP: {cm['FP']}, FN: {cm['FN']}")
# Save TP, TN, FP, FN cho từng fold vào file Excel
df_confusion = pd.DataFrame(confusion_matrices)
df_confusion.to_excel(path_result + "cv_T.gondii_KD2_Succi.xlsx", index=False)

print("\nConfusion matrix results saved to 'cv_T.gondii_KD2_Succi.xlsx'")

Training on fold 1/10...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:567: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:582: UserWarning: `model.compiled_metrics()` is deprecated. Instead, use e.g.:
```
for metric in self.metrics:
    metric.update_state(y, y_pred)
```

  return self._compiled_metrics_update_state(


48/48 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - AUC: 0.4736 - categorical_accuracy: 0.5073 - distill_loss1: 0.0359 - distill_loss2: 0.0955 - loss: 0.5000 - student_loss: 0.7038 - val_AUC: 0.6197 - val_categorical_accuracy: 0.6353 - val_loss: 0.5000 - val_student_loss: 0.6766
Epoch 2/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - AUC: 0.6849 - categorical_accuracy: 0.6319 - distill_loss1: 0.0286 - distill_loss2: 0.0822 - loss: 0.5000 - student_loss: 0.6495 - val_AUC: 0.6116 - val_categorical_accuracy: 0.6353 - val_loss: 0.5000 - val_student_loss: 0.6443
Epoch 3/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - AUC: 0.7464 - categorical_accuracy: 0.6820 - distill_loss1: 0.0254 - distill_loss2: 0.0762 - loss: 0.5000 - student_loss: 0.6234 - val_AUC: 0.6251 - val_categorical_accuracy: 0.6353 - val_loss: 0.5000 - val_student_loss: 0.6164
Epoch 4/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - AUC: 0.7488 - categorical_accuracy: 0.6779 - distill_loss1: 0.0247 - distill_loss2: 0.0702 - loss: 0.5000 - studen

...

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Fold 9 - TP: 19, TN: 38, FP: 16, FN: 11
Training on fold 10/10...


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:567: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:582: UserWarning: `model.compiled_metrics()` is deprecated. Instead, use e.g.:
```
for metric in self.metrics:
    metric.update_state(y, y_pred)
```

  return self._compiled_metrics_update_state(


48/48 ━━━━━━━━━━━━━━━━━━━━ 10s 40ms/step - AUC: 0.6361 - categorical_accuracy: 0.5815 - distill_loss1: 0.0293 - distill_loss2: 0.0868 - loss: 0.5000 - student_loss: 0.6720 - val_AUC: 0.7105 - val_categorical_accuracy: 0.6548 - val_loss: 0.5000 - val_student_loss: 0.6419
Epoch 2/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - AUC: 0.7426 - categorical_accuracy: 0.6863 - distill_loss1: 0.0262 - distill_loss2: 0.0795 - loss: 0.5000 - student_loss: 0.6383 - val_AUC: 0.7157 - val_categorical_accuracy: 0.6548 - val_loss: 0.5000 - val_student_loss: 0.6238
Epoch 3/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - AUC: 0.7200 - categorical_accuracy: 0.6741 - distill_loss1: 0.0249 - distill_loss2: 0.0747 - loss: 0.5000 - student_loss: 0.6164 - val_AUC: 0.7319 - val_categorical_accuracy: 0.6548 - val_loss: 0.5000 - val_student_loss: 0.6040
Epoch 4/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - AUC: 0.7463 - categorical_accuracy: 0.6877 - distill_loss1: 0.0255 - distill_loss2: 0.0739 - loss: 0.5000 - studen

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from tensorflow.keras.optimizers import AdamW
k = 10
kf = KFold(n_splits=k, shuffle=True, random_state=42)
all_metrics = []
confusion_matrices = []
# K-fold cross-validation
for fold, (train_index, val_index) in enumerate(kf.split(Xtrain)):
    print(f"Training on fold {fold + 1}/{k}...")
    X_train_fold, X_val_fold = Xtrain[train_index], Xtrain[val_index]
    y_train_fold, y_val_fold = ytrain[train_index], ytrain[val_index]
    teacher1_fold = create_cnn_model()
    teacher2_fold = create_bilstm_model()
    student_fold = create_student()
    teacher1_fold.compile(
        loss=keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=[
            "categorical_accuracy",
            "AUC",
        ],
        optimizer=keras.optimizers.AdamW(learning_rate=0.0001),
    )
    teacher2_fold.compile(
        loss=keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=[
            "categorical_accuracy",
            "AUC",
        ],
        optimizer=keras.optimizers.AdamW(learning_rate=0.0001),
    )
    teacher1_fold.fit(
        X_train_fold, y_train_fold,
        batch_size =16,
        epochs=50
    )

    teacher2_fold.fit(
        X_train_fold, y_train_fold,
        batch_size =16,
        epochs=50
    )
     # Re-initialize the distiller for each fold
    distiller_fold = Distiller(teacher1=teacher1_fold, teacher2=teacher2_fold, student=student_fold, alpha=0.5, temperature=10)
    distiller_fold.compile(
        student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),

        metrics=["categorical_accuracy", "AUC"],
        distillation_loss_fn=keras.losses.KLDivergence(),
    )

    # Train model
    history = distiller_fold.fit(
        X_train_fold, y_train_fold,
        validation_data=(X_val_fold, y_val_fold),
        batch_size=16,
        epochs=50,
        verbose=1
    )
    # Save result of each fold
    results = distiller_fold.evaluate(X_val_fold, y_val_fold)
    all_metrics.append(results)
    # confusion matrix và lưu TP, TN, FP, FN
    y_pred_fold = student_fold.predict(X_val_fold)
    y_pred_classes = np.argmax(y_pred_fold, axis=1)
    y_true_classes = np.argmax(y_val_fold, axis=1)
    # Tính confusion matrix và lưu TP, TN, FP, FN
    cm = confusion_matrix(y_true_classes, y_pred_classes)
    tn, fp, fn, tp = cm.ravel()
    confusion_matrices.append({"TP": tp, "TN": tn, "FP": fp, "FN": fn})
    print(f"Fold {fold + 1} - TP: {tp}, TN: {tn}, FP: {fp}, FN: {fn}")

# Average the results from the folds
avg_metrics = np.mean(all_metrics, axis=0)
print(f"\nAverage Metrics over {k}-Fold Cross Validation:")
for metric_name, avg_metric in zip(distiller_fold.metrics_names, avg_metrics):
    print(f"{metric_name}: {avg_metric:.4f}")
# Show TP, TN, FP, FN for each fold
for fold, cm in enumerate(confusion_matrices):
    print(f"Fold {fold + 1} - TP: {cm['TP']}, TN: {cm['TN']}, FP: {cm['FP']}, FN: {cm['FN']}")
#  save TP, TN, FP, FN into the file Excel
df_confusion = pd.DataFrame(confusion_matrices)
df_confusion.to_excel(path_result + "cv_T.gondii_KD2_Succ1.xlsx", index=False)

print("\nConfusion matrix results saved to 'cv_T.gondii_KD2_Succ1.xlsx'")

Training on fold 1/10...
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:567: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


48/48 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - AUC: 0.6106 - categorical_accuracy: 0.5812 - loss: 0.7867
Epoch 2/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.6314 - categorical_accuracy: 0.6181 - loss: 0.7726
Epoch 3/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.6879 - categorical_accuracy: 0.6217 - loss: 0.6956
Epoch 4/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.6024 - categorical_accuracy: 0.5569 - loss: 0.7645
Epoch 5/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.6391 - categorical_accuracy: 0.5926 - loss: 0.7402
Epoch 6/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.6584 - categorical_accuracy: 0.5985 - loss: 0.7040
Epoch 7/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.6545 - categorical_accuracy: 0.6082 - loss: 0.7030
Epoch 8/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.6577 - categorical_accuracy: 0.6232 - loss: 0.7013
Epoch 9/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.6755 - categorical_accuracy: 0.6336 - loss: 0.6849
Epoch 10/50
48/48 ━━━

/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:582: UserWarning: `model.compiled_metrics()` is deprecated. Instead, use e.g.:
```
for metric in self.metrics:
    metric.update_state(y, y_pred)
```

  return self._compiled_metrics_update_state(


48/48 ━━━━━━━━━━━━━━━━━━━━ 9s 39ms/step - AUC: 0.6038 - categorical_accuracy: 0.5505 - distill_loss1: 0.0111 - distill_loss2: 0.1125 - loss: 0.5000 - student_loss: 0.6685 - val_AUC: 0.6322 - val_categorical_accuracy: 0.6353 - val_loss: 0.5000 - val_student_loss: 0.6382
Epoch 2/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - AUC: 0.7310 - categorical_accuracy: 0.6630 - distill_loss1: 0.0072 - distill_loss2: 0.1029 - loss: 0.5000 - student_loss: 0.6262 - val_AUC: 0.6381 - val_categorical_accuracy: 0.6353 - val_loss: 0.5000 - val_student_loss: 0.6041
Epoch 3/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - AUC: 0.7417 - categorical_accuracy: 0.6841 - distill_loss1: 0.0065 - distill_loss2: 0.0964 - loss: 0.5000 - student_loss: 0.5967 - val_AUC: 0.6277 - val_categorical_accuracy: 0.6353 - val_loss: 0.5000 - val_student_loss: 0.5799
Epoch 4/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - AUC: 0.7524 - categorical_accuracy: 0.6921 - distill_loss1: 0.0064 - distill_loss2: 0.0940 - loss: 0.5000 - student

...

48/48 ━━━━━━━━━━━━━━━━━━━━ 11s 40ms/step - AUC: 0.4514 - categorical_accuracy: 0.4360 - distill_loss1: 0.0150 - distill_loss2: 0.1204 - loss: 0.5000 - student_loss: 0.7101 - val_AUC: 0.6580 - val_categorical_accuracy: 0.6429 - val_loss: 0.5000 - val_student_loss: 0.6576
Epoch 2/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - AUC: 0.6563 - categorical_accuracy: 0.6123 - distill_loss1: 0.0088 - distill_loss2: 0.1075 - loss: 0.5000 - student_loss: 0.6570 - val_AUC: 0.6746 - val_categorical_accuracy: 0.6429 - val_loss: 0.5000 - val_student_loss: 0.6274
Epoch 3/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - AUC: 0.7361 - categorical_accuracy: 0.6754 - distill_loss1: 0.0062 - distill_loss2: 0.0984 - loss: 0.5000 - student_loss: 0.6198 - val_AUC: 0.6858 - val_categorical_accuracy: 0.6429 - val_loss: 0.5000 - val_student_loss: 0.5998
Epoch 4/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - AUC: 0.7570 - categorical_accuracy: 0.6911 - distill_loss1: 0.0057 - distill_loss2: 0.0948 - loss: 0.5000 - studen

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:567: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


48/48 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - AUC: 0.5686 - categorical_accuracy: 0.5490 - loss: 0.9111
Epoch 2/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.6387 - categorical_accuracy: 0.6107 - loss: 0.7719
Epoch 3/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.5645 - categorical_accuracy: 0.5269 - loss: 0.8762
Epoch 4/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.6272 - categorical_accuracy: 0.6161 - loss: 0.7741
Epoch 5/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.6040 - categorical_accuracy: 0.5800 - loss: 0.7807
Epoch 6/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.6795 - categorical_accuracy: 0.6529 - loss: 0.6894
Epoch 7/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.5942 - categorical_accuracy: 0.5542 - loss: 0.7843
Epoch 8/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.6839 - categorical_accuracy: 0.6665 - loss: 0.6848
Epoch 9/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.6562 - categorical_accuracy: 0.6171 - loss: 0.6983
Epoch 10/50
48/48 ━━━

/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:582: UserWarning: `model.compiled_metrics()` is deprecated. Instead, use e.g.:
```
for metric in self.metrics:
    metric.update_state(y, y_pred)
```

  return self._compiled_metrics_update_state(


48/48 ━━━━━━━━━━━━━━━━━━━━ 11s 42ms/step - AUC: 0.4841 - categorical_accuracy: 0.4850 - distill_loss1: 0.0127 - distill_loss2: 0.1192 - loss: 0.5000 - student_loss: 0.7009 - val_AUC: 0.6611 - val_categorical_accuracy: 0.6548 - val_loss: 0.5000 - val_student_loss: 0.6726
Epoch 2/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - AUC: 0.6522 - categorical_accuracy: 0.6110 - distill_loss1: 0.0078 - distill_loss2: 0.1105 - loss: 0.5000 - student_loss: 0.6668 - val_AUC: 0.6702 - val_categorical_accuracy: 0.6548 - val_loss: 0.5000 - val_student_loss: 0.6391
Epoch 3/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - AUC: 0.7387 - categorical_accuracy: 0.7068 - distill_loss1: 0.0059 - distill_loss2: 0.1013 - loss: 0.5000 - student_loss: 0.6269 - val_AUC: 0.6780 - val_categorical_accuracy: 0.6548 - val_loss: 0.5000 - val_student_loss: 0.6090
Epoch 4/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - AUC: 0.7207 - categorical_accuracy: 0.6575 - distill_loss1: 0.0062 - distill_loss2: 0.0984 - loss: 0.5000 - studen